## install libraries

In [1]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/

## library import

In [22]:
import transformers
import torch
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

GLOBAL_SEED = 42

## load dataset

In [2]:
dataset = load_dataset("persiannlp/parsinlu_query_paraphrasing")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1830 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1916 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/898 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1830
    })
    test: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 1916
    })
    validation: Dataset({
        features: ['q1', 'q2', 'category', 'label'],
        num_rows: 898
    })
})

In [3]:
dataset["test"]

Dataset({
    features: ['q1', 'q2', 'category', 'label'],
    num_rows: 1916
})

In [4]:
train_df = pd.DataFrame(dataset["train"]) # for examples in one-shot and few-shot learning
test_df = pd.DataFrame(dataset["test"]) # for evaluation
test_df

,q1,q2,category,label
0,آیا جهان روح وجود دارد؟ اگر بله ، مبتکر و کنتر...,چه چیزی روح فرد را می شکند؟,qqp,0
1,چگونه می توانم تماشای فیلم های پورنو را متوقف ...,برای جلوگیری از تماشای کامل پورنو باید چه کاری...,qqp,1
2,چه کسانی امام علی را خدا میدانند؟,چه کسانی می توانند امام زمان را ببینند؟,natural,0
3,آیا قرار است دونالد ترامپ رئیس جمهور بعدی ایال...,شانس اینکه دونالد ترامپ رئیس جمهور بعدی آمریکا...,qqp,1
4,چگونه می توانم سوالی را در این باره بپرسم؟,چگونه می توانم سوال بپرسم؟,qqp,0
...,...,...,...,...
1911,چه چیزهایی روزه زن را باطل میکند اهل سنت؟,چه چیزهایی روزه زن را باطل میکند مکارم شیرازی؟,natural,0
1912,وای چه پسری وای عجب تاج سری؟,دانلود اهنگ پسر وای وای چه پسری وای عجب تاج سری؟,natural,1
1913,از چه زمانی حرکات جنین کم میشود؟,از چه زمانی حرکات جنین حس میشود؟,natural,0
1914,کدام یک از شرکتها دارای رشد شغلی بهتری دارند؟ ...,۴ لوله ورودی و ۲ لوله خروجی ، ورودی آب را پیدا...,qqp,0


## test model on a single example

In [5]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "NousResearch/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True},
    device_map="auto",
)
pipeline

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
!ls "/root/.cache/huggingface/hub/"

models--NousResearch--Meta-Llama-3-8B-Instruct	version.txt


In [58]:
!rm -rf "/root/.cache/huggingface/hub/models/models--NousResearch--Meta-Llama-3-8B-Instruct"

In [27]:
messages = [
    {"role": "system", "content": "Answer questions"}, # chat template
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer questions<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [28]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
terminators

[128001, 128009]

In [30]:
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=pipeline.tokenizer.eos_token_id
)
outputs

[{'generated_text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer questions<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text on a wide range of topics. I'm constantly learning and improving my responses based on the interactions I have with users like you!"}]

In [31]:
print(outputs[0]["generated_text"][len(prompt):])

I'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text on a wide range of topics. I'm constantly learning and improving my responses based on the interactions I have with users like you!


In [32]:
template = "Answer the questions in a single word"
prompt = f"در ادامه دو سوال آمده است. \
            آیا این دو سوال مترادف هستند یا خیر؟ \
            اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. \
            اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. \n\
            سوال اول: {train_df.iloc[100]['q1']} \n\
            سوال دوم: {train_df.iloc[100]['q2']}"

messages = [
    {"role": "system", "content": template},
    {"role": "user", "content": prompt},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=pipeline.tokenizer.eos_token_id
)

label_mapping = {
    0: "خیر",
    1: "بله"
}

print(f"Real label is: {label_mapping[int(train_df.iloc[100]['label'])]}")
model_output = outputs[0]["generated_text"][len(prompt):]
print(f"Generated text: {model_output}")

Real label is: خیر
Generated text: خیر


## inference on different scenarios

In [47]:
def create_prompt(scenario, q1, q2, examples=None):
    if scenario == "Zero-shot":
        return f"در ادامه دو سوال آمده است. \
                آیا این دو سوال مترادف هستند یا خیر؟ \
                اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. \
                اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. \n\
                سوال اول: {q1} \n\
                سوال دوم: {q2}"
    elif scenario == "One-shot":
        return f"در ادامه دو سوال آمده است. \
                آیا این دو سوال مترادف هستند یا خیر؟ \
                اگر مترادف هستند فقط در یک کلمه 'بله' را تولید کنید. \
                اگر مترادف نیستند فقط در یک کلمه 'خیر' را تولید کنید. \n\
                مثال: \n\
                سوال اول: {examples.iloc[0]['q1']} \n\
                سوال دوم: {examples.iloc[0]['q2']} \n\
                جواب: {label_mapping[int(examples.iloc[0]['label'])]}\n\
                حال با دیدن مثال جواب دهید. \n\
                سوال اول: {q1} \n\
                سوال دوم: {q2}"
    elif scenario == "Five-shot":
        examples_text = ""
        for i, example in examples.iterrows():
            examples_text += f"سوال اول: {example['q1']} \n\
                               سوال دوم: {example['q2']} \n\
                               جواب: {label_mapping[int(example['label'])]}\n"
        return f"در ادامه دو سوال آمده است. \
                آیا این دو سوال مترادف هستند یا خیر؟ \
                اگر مترادف هستند فقط در یک کلمه جواب 'بله' را تولید کنید. \
                اگر مترادف نیستند فقط در یک کلمه جواب 'خیر' را تولید کنید. \n\
                مثال ها: \n\
                {examples_text} \
                حال با دیدن مثال ها جواب دهید. \n\
                سوال اول: {q1} \n\
                سوال دوم: {q2}"

In [48]:
def generate_output_from_prompt(prompt, max_new_tokens=64):
    template = "Answer the questions in a single word"
    messages = [
        {"role": "system", "content": template},
        {"role": "user", "content": prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=pipeline.tokenizer.eos_token_id
    )
    return outputs[0]["generated_text"][len(prompt):]

In [49]:
def test_scenario(scenario, test_df, log_df, examples=None):
    correct = 0
    total = len(test_df)
    for index, row in tqdm(test_df.iterrows(), desc=f"Running {scenario} scenario..."):
        prompt = create_prompt(scenario, row['q1'], row['q2'], examples)
        output = generate_output_from_prompt(prompt)
        label = int(row['label'])
        if label == 1:
            # if output == "بله":
            # if "بله" in output.split(" ")[0]:
            if "بله" in output:
                correct += 1
        else:
            # if output == "خیر":
            # if "خیر" in output.split(" ")[0]:
            if "خیر" in output:
                correct += 1
        log_df.loc[len(log_df)] = [row['q1'], row['q2'], output, label_mapping[label], scenario]

    return correct / total

In [50]:
log_df = pd.DataFrame(columns=["q1", "q2", "generated_text", "real_label", "scenario"])

scenarios = ["Zero-shot", "One-shot", "Five-shot"]

test_subset_examples = test_df.sample(n=100, random_state=GLOBAL_SEED) # because it takes too much to test all examples we use subset of it
one_shot_subset_examples = train_df.sample(n=1, random_state=GLOBAL_SEED)
# five_shot_subset_examples = train_df.sample(n=5, random_state=GLOBAL_SEED)
# balance between 0 and 1
grouped = train_df.groupby('label')
five_shot_subset_examples = grouped.apply(lambda x: x.sample(n=2 if x.name == '0' else 3, random_state=GLOBAL_SEED))
five_shot_subset_examples = five_shot_subset_examples.reset_index(drop=True)


for scenario in scenarios:
    if scenario == "Zero-shot":
        zero_shot_result = test_scenario(scenario, test_subset_examples, log_df,examples=None)
    elif scenario == "One-shot":
        one_shot_result = test_scenario(scenario, test_subset_examples, log_df, examples=one_shot_subset_examples)
    elif scenario == "Five-shot":
        five_shot_result = test_scenario(scenario, test_subset_examples, log_df, examples=five_shot_subset_examples)
    else:
        # exception
        raise ValueError("Invalid scenario")

print("Experiment result:")
print(f"Model Name: {model_id}")
print(f"Zero-shot accuracy: {zero_shot_result}")
print(f"One-shot accuracy: {one_shot_result}")
print(f"Five-shot accuracy: {five_shot_result}")

Running Zero-shot scenario...: 100it [01:54,  1.15s/it]
Running One-shot scenario...: 100it [02:31,  1.52s/it]
Running Five-shot scenario...: 100it [04:42,  2.82s/it]

Experiment result:
Model Name: NousResearch/Meta-Llama-3-8B-Instruct
Zero-shot accuracy: 0.75
One-shot accuracy: 0.75
Five-shot accuracy: 0.4


In [51]:
# save log_df
model_name = "Meta-Llama-3-8B-Instruct"
log_df.to_csv(f"log_df_{model_name}.csv", index=False, encoding="utf-8-sig")

In [52]:
log_df

,q1,q2,generated_text,real_label,scenario
0,ويتامين ب در چه چيزهايي وجود دارد؟,ويتامين b12 در چه چيزهايي وجود دارد؟,خیر,خیر,Zero-shot
1,داستان معراج پیامبر در کدام سوره است؟,داستان معراج پیامبر در کدام سوره قرانی امده است؟,بله,بله,Zero-shot
2,کارتهای اعتباری در مشاغل کوچک و رستوران های ما...,کارتهای اعتباری در مشاغل کوچک و رستورانهای اسر...,خیر,خیر,Zero-shot
3,زیرنویس فارسی فیلم دره من چه سرسبز بود؟,دانلود زیرنویس فارسی فیلم دره من چه سرسبز بود؟,خیر,بله,Zero-shot
4,از کجا بفهمیم باتری لپ تاپ سالم است؟,از کجا بفهمیم باتری لپ تاپ خراب است؟,خیر,خیر,Zero-shot
...,...,...,...,...,...
295,چرا ندونستم دل تو رفیق نیمه راهه؟,آهنگ چرا ندونستم دل تو رفیق نیمه راهه؟,بله,بله,Five-shot
296,چه مواد غذایی برای بارداری مفید است؟,چه مواد غذایی در دوران بارداری مفید است؟,بله,خیر,Five-shot
297,چه چیزی برای از بین بردن جای جوش مفید است؟,چه چیزی برای از بین بردن جوش صورت مفید است؟,بله,خیر,Five-shot
298,چه عواملی باعث خارش چشم می شود؟,چه عواملی باعث خارش می شود؟,بله,خیر,Five-shot


In [53]:
# save the printed result in a csv file
results = {
    "Model Name": [model_id],
    "Zero-shot accuracy": [zero_shot_result],
    "One-shot accuracy": [one_shot_result],
    "Five-shot accuracy": [five_shot_result]
}

df = pd.DataFrame(results)

df.to_csv(f"experiment_results_{model_name}.csv", index=False)

In [54]:
df

,Model Name,Zero-shot accuracy,One-shot accuracy,Five-shot accuracy
0,NousResearch/Meta-Llama-3-8B-Instruct,0.75,0.75,0.4
